I REALLY want to find out if and how we can improve Kai Havertz as a progressive midfielder. This is a study on that with the very limited data I can have to myself.

In [ ]:
import pandas as pd
# Step 1: Install necessary libraries
!pip install pandas scikit-learn

# Step 2: Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score

# Adjust the file path to the location where your file is uploaded
file_path = '/content/drive/MyDrive/Football/Kai Passing.xlsx'

# Read the Excel file
df = pd.read_excel(file_path, header=1)
# Drop unnecessary columns
df = df.drop(columns=['Unnamed: 0'])
df = df.drop(columns=['Unnamed: 9'])
df = df.drop(columns=['Comp', 'Opp', 'Team', 'Player', 'Nation', 'Date', 'Pos.', 'Result', 'Min', 'Age', 'Short Cmp%','Medium Cmp%','Long Cmp%', 'Start'])
# Display the first few rows of the dataframe
def get_duplicate_columns(df):
    duplicate_column_names = set()
    for x in range(df.shape[1]):
        col = df.iloc[:, x]
        for y in range(x + 1, df.shape[1]):
            other_col = df.iloc[:, y]
            if col.equals(other_col):
                duplicate_column_names.add(df.columns[y])
    return list(duplicate_column_names)

# Step 2: Drop duplicate columns
duplicate_columns = get_duplicate_columns(df)
df = df.drop(columns=duplicate_columns)

# Display the cleaned dataframe
df.head()

,Touches Att,Touches Att 3rd,Touches Mid 3rd,Gls,Ast,G+A,G-PK,PK,PKm,Touches,...,CrsPA,PrgP,TotDist,PrgDist,Short Cmp,Short Att,Medium Cmp,Medium Att,Long Cmp,Long Att
0,127,32,99,1,0,1,1,0,0,141,...,0,9,1432,226,76,79,25,26,4,4
1,92,27,67,1,0,1,1,0,0,99,...,0,6,1075,128,48,52,27,29,1,1
2,80,24,56,1,0,1,1,0,0,86,...,0,8,863,153,43,46,17,18,2,3
3,63,17,54,0,0,0,0,0,0,72,...,0,6,711,140,36,41,16,16,1,2
4,53,9,49,1,0,1,1,0,0,64,...,0,5,632,130,23,29,13,18,3,4


In [ ]:
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

Series([], dtype: int64)


In [ ]:
X = df.drop(columns=['PrgP'])
y = df['PrgP']

# Convert categorical variables to dummy variables if needed
X = pd.get_dummies(X, drop_first=True)

# Step 5: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 6: Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_train_scaled

array([[ 0.70581621,  1.04138367,  0.31632196, ..., -0.55329211,
        -0.21618947, -0.39262015],
       [-0.75656888, -0.74409979, -0.51110148, ..., -0.71705918,
        -0.76283998, -0.87859055],
       [ 0.2879919 ,  0.09612772, -0.02843781, ...,  0.10177614,
        -0.21618947, -0.39262015],
       ...,
       [ 0.23576386,  1.04138367,  0.04051414, ...,  0.7568444 ,
        -0.21618947, -0.39262015],
       [-0.80879692, -0.00890072, -0.85586125, ..., -0.38952505,
        -0.76283998, -0.39262015],
       [ 0.13130778,  0.62126992, -0.09738976, ...,  0.10177614,
        -0.76283998, -0.87859055]])

In [ ]:
# Step 7: Train the Lasso regression model
lasso = Lasso(alpha=1.0)  # You can adjust the alpha parameter
lasso.fit(X_train_scaled, y_train)

# Step 8: Evaluate the model
y_pred_train = lasso.predict(X_train_scaled)
y_pred_test = lasso.predict(X_test_scaled)

# Training set evaluation
mse_train = mean_squared_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

# Testing set evaluation
mse_test = mean_squared_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

print(f'Training MSE: {mse_train}')
print(f'Training R^2: {r2_train}')
print(f'Testing MSE: {mse_test}')
print(f'Testing R^2: {r2_test}')

Training MSE: 2.981905847341922
Training R^2: 0.6117395296519912
Testing MSE: 4.0600687522491405
Testing R^2: 0.5300846351563494


In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the Lasso model
lasso = Lasso()

# Define the parameter grid
param_grid = {'alpha': [0.01, 0.1, 1, 10, 100]}

# Perform grid search with cross-validation
grid_search = GridSearchCV(lasso, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, y_train)

# Best parameter found by grid search
best_alpha = grid_search.best_params_['alpha']
print(f'Best alpha: {best_alpha}')

# Train the model with the best alpha
lasso_best = Lasso(alpha=best_alpha)
lasso_best.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred_train = lasso_best.predict(X_train_scaled)
y_pred_test = lasso_best.predict(X_test_scaled)

mse_train = mean_squared_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

mse_test = mean_squared_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

print(f'Training MSE: {mse_train}')
print(f'Training R^2: {r2_train}')
print(f'Testing MSE: {mse_test}')
print(f'Testing R^2: {r2_test}')

Best alpha: 0.1
Training MSE: 1.4565613013150902
Training R^2: 0.8103477423864958
Testing MSE: 2.302288623767033
Testing R^2: 0.7335314092862231


In [ ]:
# Calculate the correlation matrix
correlation_matrix = df.corr()

In [ ]:
# Extract the correlation values for PrgP
correlation_with_prgp = correlation_matrix['PrgP'].drop('PrgP')

# Sort the correlations in descending order
sorted_correlation_with_prgp = correlation_with_prgp.sort_values(ascending=False)

print("Features most positively correlated with PrgP:")
print(sorted_correlation_with_prgp.head())

print("\nFeatures most negatively correlated with PrgP:")
print(sorted_correlation_with_prgp.tail())

Features most positively correlated with PrgP:
PrgDist                    0.856623
TotDist                    0.744256
Medium Cmp                 0.723095
Medium Att                 0.713493
Passes into Final Third    0.709720
Name: PrgP, dtype: float64

Features most negatively correlated with PrgP:
G-PK               0.074293
Touches Att Pen    0.066024
CrsPA              0.041257
Ast                0.026925
PKm                     NaN
Name: PrgP, dtype: float64


In [ ]:
# Convert categorical variables to dummy variables
df_numeric = pd.get_dummies(df_cleaned, drop_first=True)

# Calculate the correlation matrix
correlation_matrix = df_numeric.corr()

# Extract the correlation values for 'PrgP'
correlation_with_prgp = correlation_matrix['PrgP'].drop('PrgP')

# Filter columns with 'Touches' in the name
touches_columns = [col for col in df_numeric.columns if 'Touches' in col]

# Get correlations for the 'Touches' columns
touches_correlations = correlation_with_prgp[touches_columns]

# Sort the correlations
sorted_touches_correlations = touches_correlations.sort_values(ascending=False)

print("Touches columns most positively correlated with PrgP:")
print(sorted_touches_correlations.head())

Touches columns most positively correlated with PrgP:
Touches Att         0.654922
Touches             0.641857
Touches Mid 3rd     0.610128
Touches  Att 3rd    0.442302
Touches  Def 3rd    0.378698
Name: PrgP, dtype: float64


In [ ]:
# Read the Excel file
df_og = pd.read_excel(file_path, header=1)
# Drop unnecessary columns
df_og = df_og.drop(columns=['Unnamed: 0'])
df_og = df_og.drop(columns=['Unnamed: 9'])

In [ ]:
# Step 3: Sort by 'Touches Mid 3rd'
df_og = df_og.sort_values(by='Touches Mid 3rd', ascending=False)
df_og = df_og[df_og['Team'] == 'Arsenal']
# Display the sorted DataFrame
print(df_og.head())  # Display the top rows

          Player  Touches Att  Touches  Att 3rd  Touches Mid 3rd       Date  \
39   Kai Havertz           54                29               34 2023-08-12   
44   Kai Havertz           41                18               34 2024-04-20   
70   Kai Havertz           50                23               28 2024-02-11   
90   Kai Havertz           30                 8               24 2023-12-31   
102  Kai Havertz           37                24               21 2024-05-04   

        Age  Nation     Team              Opp                Comp  ...  \
39   24-062  de GER  Arsenal  Nott'ham Forest  eng Premier League  ...   
44   24-314  de GER  Arsenal           Wolves  eng Premier League  ...   
70   24-245  de GER  Arsenal         West Ham  eng Premier League  ...   
90   24-203  de GER  Arsenal           Fulham  eng Premier League  ...   
102  24-328  de GER  Arsenal      Bournemouth  eng Premier League  ...   

    Short Cmp Short Att  Short Cmp%  Medium Cmp  Medium Att  Medium Cmp%  \
39  